In [1]:
import numpy as np 
import pandas as pd 
import yfinance as yf 
import datetime as dt 
from _measurements import * 
from _weight_optimization import *
from _time_varying_optimization import *
from _regime_detection import *
import seaborn as sns 
import pytse_client as tse 
import matplotlib.pyplot as plt
import os
sns.set_theme()

In [2]:
%reload_ext autoreload
%autoreload 2

In [6]:
tse.download(symbols=["وبملت", "اهرم", "شستا", "حکشتی"], write_to_csv=True)

{'اهرم':           date     open     high      low  adjClose          value     volume  \
 0   2021-12-20  10000.0  10000.0  10000.0   10000.0  4900000000000  490000000   
 1   2022-01-01  10000.0  10060.0   9999.0   10006.0    19916562444    1990468   
 2   2022-01-02  10010.0  10060.0  10001.0   10022.0    24178445323    2412421   
 3   2022-01-03  10030.0  10030.0  10000.0   10004.0    13393240845    1338832   
 4   2022-01-04  10000.0  10002.0   9970.0    9996.0    30120645975    3013175   
 ..         ...      ...      ...      ...       ...            ...        ...   
 415 2023-10-01  18490.0  18550.0  18030.0   18210.0  2212159855250  121464224   
 416 2023-10-02  18090.0  18800.0  18090.0   18450.0  2146468396810  116349307   
 417 2023-10-04  18700.0  18930.0  18570.0   18690.0  1222908790700   65447030   
 418 2023-10-07  18610.0  18610.0  17950.0   18200.0  1903999969180  104598876   
 419 2023-10-08  18100.0  18100.0  17240.0   17590.0  2808195654870  159684790   
 
      

In [7]:
returns = pd.DataFrame()
for symb in os.listdir("tickers_data"):
    data = pd.read_csv(os.path.join("tickers_data", symb))
    close = data["close"].pct_change()
    returns[symb.removesuffix(".csv")] = close
    
returns.dropna(inplace=True)
returns.head()

,اهرم,حکشتی,شستا,وبملت
1,0.006000,0.030000,0.050000,0.006667
2,-0.004076,0.029903,0.033555,-0.001892
3,-0.001896,0.005279,0.015536,0.004739
4,-0.002900,0.004876,0.026799,0.000000
5,0.002908,0.006346,0.023120,-0.008491


In [4]:
# # List of symbols
# symbols = [
#     'AAPL', 'MSFT', 'GOOGL', 
#     'AMZN', "TSLA", 'NVDA', 
#     'JNJ', 'WMT', 'KO', 
#     'BAC', 'ADBE', 'MCD',
#     'NFLX', 'NKE', 'PUM.DE',
#     'PM', 'IBM', 'GE', "GM"
# ]

# # Download historical data for the symbols
# data = yf.download(symbols, end='2023-08-07')["Close"]

# # Print the first few rows of the data
# data.dropna(inplace=True)
# returns = data.pct_change().dropna()
# returns

In [6]:
# methods_mu = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
# methods_cov = ["historical", "EWMA", "ledoit", "oracle", "shrunk", "mincovdet"]
# objectives = ["min_risk", "max_sharpe", "max_return", "max_utility"]

# final_results = []

# for i in range(len(methods_mu)):
#     mu = methods_mu[i]
#     cov = methods_cov[i]
#     for objective in objectives:
#         params = {
#             "train_low": 30, 
#             "train_high": 33, 
#             "test_low": 30, 
#             "test_high": 33, 
#             "step": 1,
#             "method_mu": mu, 
#             "method_cov": cov, 
#             "objective": objective
#             }
        
#         int_opt = RollingTimeOptimization(returns, params)
#         results = int_opt()
#         final_results.append(results)

In [10]:
data = (returns + 1).cumprod()
rg = RegimeDetection(data)
betas = rg.trend_filtering(returns*100, lambda_value=12)
tr = rg.regime_switch_series(betas)

         اهرم     حکشتی      شستا     وبملت
1   -0.220630  1.353377  3.793733  0.154160
2   -0.220630  1.353377  3.793733  0.154160
3   -0.220630  1.353377  3.793733  0.154160
4   -0.220630  1.353377  3.793733  0.154160
5   -0.220630  1.353377  3.793733  0.154160
..        ...       ...       ...       ...
415 -0.604973 -0.448723 -0.241362 -0.075348
416 -0.604973 -0.448723 -0.241362 -0.075348
417 -0.604973 -0.448723 -0.241362 -0.075348
418 -0.604973 -0.189379 -0.241362 -0.075348
419 -0.604973 -0.189379 -0.241362 -0.075348

[419 rows x 4 columns]


In [16]:
import plotly.graph_objects as go

symbol = "شستا"
fig = go.Figure()

# Fill bullish regions
fig.add_trace(go.Scatter(
    x=data.index,
    y=[np.max(data[symbol]) if t > 0 else 0 for t in tr[symbol]],
    fill='tozeroy',
    mode='none',
    fillcolor='lightgreen',
    name='bullish'
))

# Fill bearish regions
fig.add_trace(go.Scatter(
    x=data.index,
    y=[np.max(data[symbol]) if t < 0 else 0 for t in tr[symbol]],
    fill='tozeroy',
    mode='none',
    fillcolor='salmon',
    name='bearish'
))

# Plot stock price
fig.add_trace(go.Scatter(
    x=data.index,
    y=data[symbol],
    mode='lines',
    name='Stock Price',
    line=dict(color='blue')
))

fig.show()